In [1]:
import splusdata
from tqdm import tqdm

import numpy as np
from astropy.io import fits

from src.auxiliar import plot_fits
from src.get_data import gen_dataset
from src.log import control

In [2]:
gen_dataset("""
            SELECT 
            TOP 2000
            det.id, det.ra, det.dec, det.field, det.B, det.A, det.flux_radius_50, det.theta, det.fwhm,
            
            r.r_auto
            FROM "idr4_dual"."idr4_detection_image" AS det 
            
            LEFT OUTER JOIN "idr4_dual"."idr4_dual_r" AS r ON r.id = det.id
            
            WHERE 1 = CONTAINS( POINT('ICRS', det.ra, det.dec), 
                    CIRCLE('ICRS', 0.5, 0.4, 1) ) 
                    AND r.r_auto < 21
    """,
    fits_folder="data/",
    arrays_folder="arrays/"
    )

10/03/2024 02:08:19  [info] - get_data.py - gen_dataset() - Insert splus.cloud credentials
10/03/2024 02:08:24  [info] - get_data.py - gen_dataset() - Querying data
10/03/2024 02:08:36  [info] - get_data.py - gen_dataset() - Downloading stamps
10/03/2024 02:08:36  [info] - get_data.py - threadable_splus_stamp() - Stamp for iDR4_3_STRIPE82-0002_0000151 downloaded. 1/2000
10/03/2024 02:08:41  [info] - get_data.py - threadable_splus_stamp() - Stamp for iDR4_3_STRIPE82-0002_0000166 downloaded. 2/2000
10/03/2024 02:08:41  [info] - get_data.py - threadable_splus_stamp() - Stamp for iDR4_3_STRIPE82-0002_0000209 downloaded. 3/2000
10/03/2024 02:08:41  [info] - get_data.py - threadable_splus_stamp() - Stamp for iDR4_3_STRIPE82-0002_0000195 downloaded. 4/2000
10/03/2024 02:08:41  [info] - get_data.py - threadable_splus_stamp() - Stamp for iDR4_3_STRIPE82-0002_0000244 downloaded. 5/2000
10/03/2024 02:08:41  [info] - get_data.py - threadable_splus_stamp() - Stamp for iDR4_3_STRIPE82-0002_0000197 d

In [2]:
conn = splusdata.Core()

In [3]:
df = conn.query(f"""
        SELECT 
        TOP 2000
        det.id, det.ra, det.dec, det.field, det.B, det.A, det.flux_radius_50, det.theta, det.fwhm,
        
        r.r_auto
        FROM "idr4_dual"."idr4_detection_image" AS det 
        
        LEFT OUTER JOIN "idr4_dual"."idr4_dual_r" AS r ON r.id = det.id
        
        WHERE 1 = CONTAINS( POINT('ICRS', det.ra, det.dec), 
				CIRCLE('ICRS', 0.5, 0.4, 1) ) 
				AND r.r_auto < 21
        """)

df = df.to_pandas()

In [69]:
df["ID"][0]

'iDR4_3_STRIPE82-0002_0000151'

In [ ]:
try:
    df['ID'] = df['ID'].apply(lambda x: x.decode('utf-8'))
except:
    pass 

df.to_csv('data.csv', index=False)

In [37]:
stamps = []

for key, value in tqdm(df.iterrows()):
    stamp = conn.stamp(value["RA"], value["DEC"], 250, "R")
    stamps.append(stamp)

2000it [24:46,  1.35it/s]


In [51]:
for key, value in tqdm(df.iterrows()):
    stamps[key].writeto(f"data/{value['ID'].decode('utf-8')}.fits.fz", overwrite=True)

0it [00:00, ?it/s]

2000it [00:15, 128.62it/s]


In [62]:
df['FWHM'].to_numpy()

array([0.        , 0.        , 0.        , ..., 0.00055736, 0.00044423,
       0.00029422], dtype=float32)